##  Week3 복습과제
### 
#### 데이터셋(train)을 다운 받아주세요

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#### 'train.csv'을 train_df에 저장하시오

In [6]:
# 답
train_df = pd.read_csv(r'C:\temp\train_b.csv')

#### 데이터 보정을 위해 다음 코드를 실행시켜 주세요.

In [7]:
# id 제외

train_df = train_df.drop("id", axis = 1)

In [8]:
# color, type encoding 
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

encoder= LabelEncoder()

train_df['color'] = encoder.fit_transform(train_df['color'])
train_df['type'] = encoder.fit_transform(train_df['type'])

train_df.head()

,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0.354512,0.350839,0.465761,0.781142,3,1
1,0.575560,0.425868,0.531401,0.439899,4,2
2,0.467875,0.354330,0.811616,0.791225,0,1
3,0.776652,0.508723,0.636766,0.884464,0,1
4,0.566117,0.875862,0.418594,0.636438,4,0


#### 독립변수(X)와 종속변수(y)로 분리해주세요(iloc 또는 loc 활용) 그리고 확인해주세요 이때 target은 type입니다
#### X와 y를 X_train, X_test, y_train, y_test으로 나누시오. (test_size=0.2, random_state=12) 

In [9]:
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [10]:
# 답

X = train_df.iloc[:,:-1]
y = train_df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

#### Gradient Boost를 활용하여 train 세트를 fit 한 후 test 세트로 예측의 정확도를 출력하시오. (random_state=0,n_estimators=50,learning_rate=0.5)

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix, accuracy_score

In [12]:
#답

gb_clf = GradientBoostingClassifier(random_state=0,n_estimators=50,learning_rate=0.5)
gb_clf.fit(X_train,y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test,gb_pred)

print('GBM 정확도:{0:.6f}'.format(gb_accuracy))

GBM 정확도:0.706667


### GBM 하이퍼 파라미터 튜닝을 한 후, 최적의 하이퍼 파라미터를 구하고 정확도를 출력하시오. 위의 값과 정확도를 비교하시오. 이때 n_estimators는 50,100,500, learning_rate는 0.005, 0.1, 0.5,1,10 을 변수로 넣으시오.(gridsearchCV이용,cv=2,verbose=1)

In [13]:
# 답
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[50,100,500],
    'learning_rate' : [0.05,0.1,0.5,1,10]
}

grid_cv = GridSearchCV(gb_clf,param_grid=params,cv=2,verbose=1)
grid_cv.fit(X_train,y_train)
print('최적의 하이퍼 파라미터:\n',grid_cv.best_params_)

Fitting 2 folds for each of 15 candidates, totalling 30 fits
최적의 하이퍼 파라미터:
 {'learning_rate': 0.1, 'n_estimators': 100}


In [14]:
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test,gb_pred)
print('GBM정확도:{0:.6f}'.format(gb_accuracy))

GBM정확도:0.733333


정확도가 0.70 -> 0.73 으로 파라미터 최적화 후 상승함을 볼 수 있다

### 사이킷런 래퍼 클래스 XGBoost를 사용해 train 세트를 fit 한 후 test 세트로 예측의 정확도를 출력하시오. (n_estimators=400, learning_rate = 0.1, max_depth = 3)

In [15]:
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
warnings.filterwarnings('ignore')

In [16]:
# 답

xgb_wrapper = XGBClassifier(n_estimators=400,
                            max_depth=3,
                            learning_rate=0.1)
xgb_wrapper.fit(X_train, y_train)
pred = xgb_wrapper.predict(X_test)

print(accuracy_score(y_test, pred))

0.68


### 사이킷런 래퍼 클래스 LightGBM를 사용해 train 세트를 fit 한 후 test 세트로 예측의 정확도를 출력하시오. 
#### n_estimators=400
#### 조기 중단 파라미터: early_stopping_rounds=100, eval_metric="logloss",  eval_set=evals 

In [17]:
import lightgbm
from lightgbm import LGBMClassifier

evals = [(X_test, y_test)]

In [18]:
# 답

lgbm_wrapper = LGBMClassifier(n_estimators=400)

lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="logloss", 
                 eval_set=evals , verbose=True)
preds = lgbm_wrapper.predict(X_test)

[1]	valid_0's multi_logloss: 1.04658
[2]	valid_0's multi_logloss: 0.991864
[3]	valid_0's multi_logloss: 0.961098
[4]	valid_0's multi_logloss: 0.922715
[5]	valid_0's multi_logloss: 0.889621
[6]	valid_0's multi_logloss: 0.860335
[7]	valid_0's multi_logloss: 0.838603
[8]	valid_0's multi_logloss: 0.823945
[9]	valid_0's multi_logloss: 0.808374
[10]	valid_0's multi_logloss: 0.794635
[11]	valid_0's multi_logloss: 0.787051
[12]	valid_0's multi_logloss: 0.779236
[13]	valid_0's multi_logloss: 0.772983
[14]	valid_0's multi_logloss: 0.764275
[15]	valid_0's multi_logloss: 0.752906
[16]	valid_0's multi_logloss: 0.746936
[17]	valid_0's multi_logloss: 0.748563
[18]	valid_0's multi_logloss: 0.744367
[19]	valid_0's multi_logloss: 0.743001
[20]	valid_0's multi_logloss: 0.73754
[21]	valid_0's multi_logloss: 0.73435
[22]	valid_0's multi_logloss: 0.730277
[23]	valid_0's multi_logloss: 0.729156
[24]	valid_0's multi_logloss: 0.727211
[25]	valid_0's multi_logloss: 0.73012
[26]	valid_0's multi_logloss: 0.729092

In [19]:
print(accuracy_score(y_test , preds))

0.68


## Stacking 실습

CV set 기반 stacking을 통해 앙상블 모델을 만드려고 한다.  
개별 모델이 교차검증을 통해 최종 모델이 사용할 학습용, 테스트용 데이터를 만드는 함수를 생성하시오.
```
def get_stacking_dataset(model, X_train, y_train, X_test, n_folds ):
    return train_fold_pred , test_pred_mean
```

In [20]:
from sklearn.model_selection import KFold

def get_stacking_dataset(model, X_train, y_train, X_test, n_folds):

  kfold = KFold(n_splits=n_folds)
  train_fold_pred = np.zeros((X_train.shape[0], 1))
  test_pred_mean = np.zeros((X_test.shape[0], n_folds))

  for fold, (train_idx, valid_idx) in enumerate(kfold.split(X_train)):
    X_tr = X_train.loc[train_idx]
    y_tr = y_train.loc[train_idx]
    X_val = X_train.loc[valid_idx]

    model.fit(X_tr, y_tr)
    train_fold_pred[valid_idx, :] = model.predict(X_val).reshape(-1, 1)
    test_pred_mean[:, fold] = model.predict(X_test)

  test_pred_mean = np.mean(test_pred_mean, axis=1).reshape(-1,1)
  
  return train_fold_pred, test_pred_mean

스태킹에 사용될 개별 모델을 생성하시오.

개별 모델:  
KNN(n_neighbors=4), 랜덤 포레스트(n_estimators=100, random_state=0), 결정 트리, AdaBoost(n_estimators=100), LightGBM(n_estimators=400)

최종 모델: 로지스틱 회귀 (C=10)

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

knn_clf  = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)
lgbm_clf = LGBMClassifier(n_estimators=400)
lr_final = LogisticRegression(C=10)

위에서 생성한 함수를 사용해 메타모델에 사용될 학습용 데이터와 테스트용 데이터를 생성하고, 스태킹하시오.  
fold 수 = 6  
학습용 데이터 : stacked_X_train  
테스트용 데이터 : stacked_X_test

In [22]:
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

In [23]:
knn_train, knn_test = get_stacking_dataset(knn_clf, X_train, y_train, X_test, n_folds= 6)
rf_train, rf_test = get_stacking_dataset(rf_clf, X_train, y_train, X_test, 6)
dt_train, dt_test = get_stacking_dataset(dt_clf, X_train, y_train, X_test,  6)    
ab_train, ab_test = get_stacking_dataset(ada_clf, X_train, y_train, X_test, 6)
lgbm_train, lgbm_test = get_stacking_dataset(lgbm_clf, X_train, y_train, X_test, 6)

In [24]:
stacked_X_train = np.concatenate((knn_train, rf_train, dt_train, ab_train, lgbm_train), axis=1)
stacked_X_test = np.concatenate((knn_test, rf_test, dt_test, ab_test, lgbm_test), axis=1)

최종 모델을 학습시키고, 정확도를 구하시오.

In [25]:
lr_final.fit(stacked_X_train, y_train)
pred_final = lr_final.predict(stacked_X_test)
print(accuracy_score(y_test, pred_final))

0.6266666666666667


### CatBoost

범주형 변수가 많은 데이터에 catboost를 적용해봅시다.

데이터 출처:
https://www.kaggle.com/datasets/uciml/mushroom-classification


In [27]:
mushroom_df = pd.read_csv(r'C:\temp\mushrooms.csv')

mushroom_df feature들의 data type을 확인하시오.

In [28]:
mushroom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

feature와 target을 각각 X_mushroom, y_mushroom으로 분리하고,  
train_test_split 함수로 train, test set을 분리하시오. (test size = 0.2)

In [29]:
X_mushroom = mushroom_df.iloc[:, 1:]
y_mushroom = mushroom_df.iloc[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X_mushroom, y_mushroom, test_size=0.2)

CatBoostClassifier를 불러오고, Pool 함수를 이용해 범주형 변수를 처리하도록 train_pool, test_pool을 만드시오.

In [30]:
!pip install catboost

In [31]:
from catboost import CatBoostClassifier, Pool

In [32]:
train_pool = Pool(X_train, y_train, cat_features=list(X_train.columns))
test_pool = Pool(X_test, y_test, cat_features=list(X_test.columns))

In [33]:
from sklearn.metrics import accuracy_score

iteration=400, learning_rate=0.1, max_depth=3로 설정한 CatBoost 모델로 버섯에 대한 정보를 학습하고 테스트 데이터 버섯의 독성 여부를 예측하시오. 또 정확도를 구하시오.  

In [34]:
cb_clf = CatBoostClassifier(iterations=400, learning_rate=0.1, max_depth=3)
cb_clf.fit(train_pool)
pred = cb_clf.predict(test_pool)

print(accuracy_score(y_test, pred))

0:	learn: 0.4815189	total: 178ms	remaining: 1m 10s
1:	learn: 0.3451069	total: 197ms	remaining: 39.1s
2:	learn: 0.2400314	total: 215ms	remaining: 28.5s
3:	learn: 0.1835874	total: 233ms	remaining: 23s
4:	learn: 0.1369710	total: 250ms	remaining: 19.7s
5:	learn: 0.0957000	total: 271ms	remaining: 17.8s
6:	learn: 0.0788831	total: 288ms	remaining: 16.2s
7:	learn: 0.0624718	total: 304ms	remaining: 14.9s
8:	learn: 0.0521032	total: 340ms	remaining: 14.8s
9:	learn: 0.0393197	total: 361ms	remaining: 14.1s
10:	learn: 0.0350637	total: 381ms	remaining: 13.5s
11:	learn: 0.0311706	total: 400ms	remaining: 12.9s
12:	learn: 0.0294368	total: 421ms	remaining: 12.5s
13:	learn: 0.0272894	total: 440ms	remaining: 12.1s
14:	learn: 0.0255361	total: 456ms	remaining: 11.7s
15:	learn: 0.0238433	total: 476ms	remaining: 11.4s
16:	learn: 0.0219074	total: 496ms	remaining: 11.2s
17:	learn: 0.0206377	total: 515ms	remaining: 10.9s
18:	learn: 0.0191577	total: 536ms	remaining: 10.7s
19:	learn: 0.0190245	total: 552ms	remainin